In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/Encoder-Decoder-JCLdic/experiments

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
!nvidia-smi

In [ ]:
!pip install OpenNMT-py

In [ ]:
df = pd.read_pickle('../data/jcl_full_preprocessed.pickle')
df = df.dropna()

df['src'] = df['c0'].apply(lambda x: ' '.join(list(x)))
df['tgt'] = df['c10'].apply(lambda x: ' '.join(list(x)))
y_dummy = range(len(df))

train, test, _, y_dummy = train_test_split(df, y_dummy, test_size=0.4, shuffle=False, random_state=42)
valid, test, _, _ = train_test_split(test, y_dummy, test_size=0.5, shuffle=False, random_state=42)

train['src'].to_csv('../data/train-src.txt', index=False, header=False)
valid['src'].to_csv('../data/valid-src.txt', index=False, header=False)
test['src'].to_csv('../data/test-src.txt', index=False, header=False)
train['tgt'].to_csv('../data/train-tgt.txt', index=False, header=False)
valid['tgt'].to_csv('../data/valid-tgt.txt', index=False, header=False)
test['tgt'].to_csv('../data/test-tgt.txt', index=False, header=False)

In [ ]:
!onmt_preprocess  \
  -train_src ../data/train-src.txt  \
  -train_tgt ../data/train-tgt.txt  \
  -valid_src ../data/valid-src.txt  \
  -valid_tgt ../data/valid-tgt.txt  \
  -save_data ../data/data  \
  -src_vocab_size 10000  \
  -tgt_vocab_size 10000

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
!onmt_train  \
  -data ../data/data  \
  -save_model ../data/demo-model  \
  -train_steps 10000  \
  -world_size 1  \
  -gpu_ranks 0

In [ ]:
!onmt_translate  \
  -model ../data/demo-model_step_10000.pt  \
  -src ../data/test-src.txt  \
  -output ../data/test-pred.txt  \
  -replace_unk -verbose